In [9]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [10]:
!pip install gensim==3.8.3

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 23.4/23.4 MB 282.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for gensim
Failed to build gensim
  Running setup.py install for gensim: started
  Running setup.py install for gensim: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [636 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\gensim
  copying gensim\downloader.py -> build\lib.win-amd64-cpython-310\gensim
  copying gensim\interfaces.py -> build\lib.win-amd64-cpython-310\gensim
  copying gensim\matutils.py -> build\lib.win-amd64-cpython-310\gensim
  copying gensim\nosy.py -> build\lib.win-amd64-cpython-310\gensim
  copying gensim\utils.py -> build\lib.win-amd64-cpython-310\gensim
  copying gensim\__init__.py -> build\lib.win-amd64-cpython-310\gensim
  creating build\lib.win-amd64-cpython-310\gensim\corpora
  copying gensim\corpora\bleicorpus.py -> build\lib.win-amd64-cpython-310\gensim\corpora
  copying gensim\corpora\csvcorpus.py -> build\lib.win-amd64-cpython-310\gensim\corpora
  copying gensim\corpora\dictionary.

In [17]:

train_df = pd.read_csv("C:\\Users\\aitma\\Downloads\\quora_duplicate_questions.tsv",sep='\t')

#this path concern a binary file that contains a corpus of words with its embedding vectors 
#such as we get these vectors trough a trained model that embedd words into vectors to extract similarities between these words

EMBEDDING_FILE = "C:\\Users\\aitma\\Desktop\\GoogleNews-vectors-negative300.bin.gz"

stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# Prepare embedding
vocabulary = dict()
inverse_vocabulary = []  
#KeyedVectors is a class that contains functions allowing us to interact with a bainiries files getted 
#from word2vec model so we load this binary file . 
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
#In this example, iterrows() is called on the DataFrame df. It returns an iterator that
# yields pairs of index and row data. 
# Inside the loop, you can access the index and values of each row using
# the index and row variables, respectively.
for dataset in [train_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.key_to_index.keys():
                    continue
                #so here we try to collect unique words of each question such as each word has its own index
                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)+1
                    #q2n list contains indexes of all unique words of  iterated questions
                    q2n.append(len(inverse_vocabulary)+1)
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.at[index, question]=q2n
            
embedding_dim = 300
#here we try to generate a random embedding matrix where on each row there is embedding vector of a word 

embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix


# Build the embedding matrix through the 
for word, index in vocabulary.items():
    if word in word2vec.key_to_index.keys():
        embeddings[index] = word2vec.get_vector(word)

del word2vec

In [18]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10]",0
1,1,3,4,"[1, 2, 3, 12, 13, 14, 15, 16, 15, 17, 18]","[1, 19, 20, 21, 3, 22, 23, 24, 3, 13, 14, 15, ...",0
2,2,5,6,"[26, 27, 16, 28, 3, 29, 30, 31, 32, 33, 34, 35]","[26, 27, 31, 29, 36, 37, 5, 38, 39, 40]",0
3,3,7,8,"[41, 42, 16, 43, 44, 45, 26, 27, 16, 46, 47]","[48, 3, 49, 50, 51, 52, 53, 54, 51, 2, 55, 5, ...",0
4,4,9,10,"[56, 57, 58, 8, 59, 60, 61, 62, 63, 64, 65, 66]","[56, 67, 19, 68, 8, 62, 59]",0


In [19]:
inverse_vocabulary[0]

'what'

In [20]:
np.shape(embeddings)

(86002, 300)

In [21]:
q2n

[1, 2, 47, 139, 401, 2543, 175, 135, 7229]

In [22]:
inverse_vocabulary

['what',
 'is',
 'the',
 'step',
 'by',
 'guide',
 'invest',
 'in',
 'share',
 'market',
 'india',
 'story',
 'kohinoor',
 'koh',
 '-',
 'i',
 'noor',
 'diamond',
 'would',
 'happen',
 'if',
 'indian',
 'government',
 'stole',
 'back',
 'how',
 'can',
 'increase',
 'speed',
 'my',
 'internet',
 'connection',
 'while',
 'using',
 'vpn',
 'be',
 'increased',
 'hacking',
 'through',
 'dns',
 'why',
 'am',
 'mentally',
 'very',
 'lonely',
 'solve',
 'it',
 'find',
 'remainder',
 'when',
 'math',
 '23',
 '^',
 '24',
 'divided',
 'which',
 'one',
 'dissolve',
 'water',
 'quikly',
 'sugar',
 'salt',
 'methane',
 'carbon',
 'di',
 'oxide',
 'fish',
 'survive',
 'astrology',
 ':',
 'capricorn',
 'sun',
 'cap',
 'moon',
 'rising',
 'does',
 'that',
 'say',
 'about',
 'me',
 'triple',
 'ascendant',
 'this',
 'should',
 'buy',
 'tiago',
 'keeps',
 'childern',
 'active',
 'far',
 'from',
 'phone',
 'video',
 'games',
 'good',
 'geologist',
 'do',
 'great',
 'you',
 'use',
 'instead',
 'motorola',
 

In [23]:
train_df[questions_cols]

,question1,question2
0,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10]"
1,"[1, 2, 3, 12, 13, 14, 15, 16, 15, 17, 18]","[1, 19, 20, 21, 3, 22, 23, 24, 3, 13, 14, 15, ..."
2,"[26, 27, 16, 28, 3, 29, 30, 31, 32, 33, 34, 35]","[26, 27, 31, 29, 36, 37, 5, 38, 39, 40]"
3,"[41, 42, 16, 43, 44, 45, 26, 27, 16, 46, 47]","[48, 3, 49, 50, 51, 52, 53, 54, 51, 2, 55, 5, ..."
4,"[56, 57, 58, 8, 59, 60, 61, 62, 63, 64, 65, 66]","[56, 67, 19, 68, 8, 62, 59]"
...,...,...
404285,"[26, 184, 3632, 115, 307, 8, 3, 24591, 522, 52...","[26, 184, 3632, 115, 307, 8, 12037, 522, 523, ..."
404286,"[97, 99, 2441, 307, 2, 598, 180, 1822]","[2, 47, 467, 77, 307, 2, 598, 180, 1822]"
404287,"[1, 2, 57, 11017]","[1, 2, 83, 11017]"
404288,"[1, 2, 3, 21210, 12592, 534, 2769, 33, 3114, 8...","[16, 42, 1086, 2877, 2854, 2622, 1220, 16, 173..."


In [25]:
#here we try to get the length of longest question in dataset to unify the dimension vectors of all the words for calculating 

max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max())

# Split to train validation
validation_size = 1000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

# Split to dicts
X_train = {'question1': X_train.question1, 'question2': X_train.question2}
X_validation = {'question1': X_validation.question1, 'question2': X_validation.question2}


# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values


for key in ['question1','question2']:
    X_train[key]=pad_sequences(X_train[key], maxlen=max_seq_length)
    X_validation[key]=pad_sequences(X_validation[key], maxlen=max_seq_length)

# Make sure everything is ok
boolean1=len(X_train['question1']) == len(Y_train)
boolean2=X_train['question1'].shape == X_train['question2'].shape
print(boolean1,boolean2)

True True


In [26]:
X_train

{'question1': array([[    0,     0,     0, ...,   180,   335, 18316],
        [    0,     0,     0, ...,   340, 76552,  4654],
        [    0,     0,     0, ...,   175,     3,  3457],
        ...,
        [    0,     0,     0, ...,   167,   483,    41],
        [    0,     0,     0, ...,  1686,   108,  6676],
        [    0,     0,     0, ...,  3635,   290,  1611]]),
 'question2': array([[    0,     0,     0, ...,     3,   136,  1913],
        [    0,     0,     0, ..., 22211, 22615,  7533],
        [    0,     0,     0, ...,     3,   321,   230],
        ...,
        [    0,     0,     0, ...,     1,    99,   173],
        [    0,     0,     0, ...,  1221,   169,  6676],
        [    0,     0,     0, ...,   988,  1613,  2165]])}

In [27]:
X_validation

{'question1': array([[   0,    0,    0, ..., 1535, 6162, 2900],
        [   0,    0,    0, ...,   97, 1174,  715],
        [   0,    0,    0, ...,   77, 1261,  815],
        ...,
        [   0,    0,    0, ...,  310,    8,  768],
        [   0,    0,    0, ...,  115,  148, 2242],
        [   0,    0,    0, ...,  182,  271, 2543]]),
 'question2': array([[    0,     0,     0, ...,   129,     3,   228],
        [    0,     0,     0, ...,    76,   503,   715],
        [    0,     0,     0, ...,   421,   815, 11059],
        ...,
        [    0,     0,     0, ..., 10054,     8,   768],
        [    0,     0,     0, ...,    97,   183, 27104],
        [    0,     0,     0, ...,   182,     8,  2543]])}

In [28]:
max_seq_length

212

In [29]:
# Model variables
n_hidden = 50

batch_size = 60
n_epoch = 3

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    #keepdims=True : This means that the shape of the resulting array will have the same number
    #of dimensions as the input array,
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')
# the trainable=False setting ensures that the pre-trained word embeddings 
#are not modified during model training.

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)


left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)
'''We have two embedded matrices that represent a candidate of two similar questions. Then we feed them into the
LSTM (practically, there is only one) and the final state of the LSTM for each question is a 50-dimensional vector. 
It is trained to capture semantic meaning of the question.
In figure 1, this vector is denoted by the letter h.'''

# Calculates the distance as defined by the MaLSTM model
#Lambda is used to transform the input data using an expression or function. For example, if Lambda with expression lambda x: x ** 2 is applied to a layer,
#then its input data will be squared before processing. function represent the lambda function.
# x=[left_output,right_output] 
#and x[0]=left_ouptut  x[0][0]=50
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
#model = Model(input_layer, output_layer)

malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
#optimizer=optimizer
malstm.compile(loss='mean_squared_error',optimizer=Adam(), metrics=['accuracy'])

# Start training
training_start_time = time()
#Note :    the test data has not any relation with bathc_size and n_epochs
malstm_trained = malstm.fit([X_train['question1'], X_train['question2']], Y_train, batch_size=batch_size, epochs=n_epoch,
                            validation_data=([X_validation['question1'], X_validation['question2']], Y_validation))
'''Each epoch consists of one forward pass (input data is fed to the model) and one backward pass 
(gradient computation and weight updates). 
Increasing the number of epochs allows the model 
to see the training data multiple times and potentially improve its performance.'''

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

Epoch 1/3
6722/6722 [==============================] - 1981s 294ms/step - loss: 0.1586 - accuracy: 0.7740 - val_loss: 0.1438 - val_accuracy: 0.7950
Epoch 2/3
6722/6722 [==============================] - 2100s 312ms/step - loss: 0.1389 - accuracy: 0.8063 - val_loss: 0.1406 - val_accuracy: 0.7990
Epoch 3/3
6722/6722 [==============================] - 1819s 271ms/step - loss: 0.1316 - accuracy: 0.8187 - val_loss: 0.1380 - val_accuracy: 0.8120
Training time finished.
3 epochs in 1:38:20.359323


In [50]:
#for some manually validation
new_text = "What is the main idea conveyed in the book?"
same_text = " How does the central message of the book impact the overall storyline?"
dif_text = "What is the role of international diplomacy in resolving global conflicts and promoting peace?"
    


In [51]:
test_df = pd.DataFrame(np.array([[new_text, same_text], [new_text, dif_text]]),
                   columns=['question1', 'question2'])

text1_preprocess = test_df['question1'].apply(text_to_word_list)
text2_preprocess = test_df['question2'].apply(text_to_word_list)

t1v = list()
t2v = list()

In [52]:
for row_text1 in text1_preprocess:
    t1_num = []
    for word in row_text1 :
        if word in vocabulary:
            t1_num.append(vocabulary[word])
    t1v.append(t1_num)

for row_text2 in text2_preprocess:
    t2_num = []
    for word in row_text2: 
        if word in vocabulary:
            t2_num.append(vocabulary[word])
    t2v.append(t2_num)

In [53]:
df_new = pd.DataFrame(zip(t1v, t2v), columns=["question1", "question2"])
df_new

,question1,question2
0,"[1, 2, 3, 2335, 1630, 19488, 8, 3, 508]","[26, 76, 3, 3064, 1932, 3, 508, 1255, 3, 5710,..."
1,"[1, 2, 3, 2335, 1630, 19488, 8, 3, 508]","[1, 2, 3, 2989, 157, 22429, 8, 44834, 2439, 27..."


In [54]:
X_test = {'left': df_new.question1, 'right': df_new.question2}
for key in ['left','right']:
    X_test[key]=pad_sequences(X_test[key], maxlen=max_seq_length)
assert X_test['left'].shape == X_test['right'].shape    


In [55]:
result = malstm.predict([X_test['left'], X_test['right']])

1/1 [==============================] - 0s 65ms/step


In [56]:
result

array([[0.3737058 ],
       [0.24643417]], dtype=float32)

We got 0.37 for the distance of similar question and 0.24 between two diffirent question